In [ ]:
# Pick up here

# Resources
   # https://www.youtube.com/watch?v=d2wZPFKAFjc
   # https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/automated-machine-learning/forecasting-orange-juice-sales

In [6]:
import pandas as pd
df = pd.read_csv('vending_revenue_forecast2.csv')

In [9]:
df.to_csv('converted_dates.csv', index=False)

In [65]:
# Preprocessing the raw data

"""
import pandas as pd

# Preprocessing
df = pd.read_csv('vend.csv').dropna()
txn = df.value_counts('Transaction')
txn = pd.DataFrame(txn, columns=['Basket']).reset_index()
df = df.merge(txn, how='left', on='Transaction')
df.sort_values('Basket', ascending=False)
df = df[['Transaction', 'TransDate', 'Machine', 'Product', 'Type', 'RCoil', 'RPrice', 'RQty', 'LineTotal', 'TransTotal', 'Basket']]
df_txn = pd.read_csv('unique.csv')
df.merge(df_txn, how='left', on='Product')
df = df[['Transaction', 'Date', 'Machine', 'Product', 'Type', 'Category', 'Healthy', 'Payment', 'Coil', 'Price', 'Qty', 'LineTotal', 'TransTotal', 'BasketSize']]
df = df.to_csv('vending_machine.csv')
# Manual edits, save as vending.csv

df['DayOfWeek'] = pd.to_datetime(df['Date'], format='%m/%d/%y').dt.strftime('%a')


from datetime import datetime, timedelta

# Convert 'Date' column to datetime format
df['Date2'] = pd.to_datetime(df['Date'], format='%m/%d/%y')

# Function to get the week of the year starting on Saturday
def get_week_of_year(date):
    year_start = datetime(date.year, 1, 1)
    days_offset = (6 - year_start.weekday()) % 7  # Offset to adjust for starting on Saturday
    year_start += timedelta(days=days_offset)
    week_number = (date - year_start).days // 7 + 1
    return week_number

# Add a new column for week of the year
df['WeekOfYear'] = df['Date2'].apply(get_week_of_year)

pd.get_dummies(df, columns=['Type', 'Category', 'HealthyMarketing', 'Payment', 'DayOfWeek])

# Add most recent Saturday
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%y')
offset = (df['Date'].dt.dayofweek + 2) % 7
most_recent_saturday = pd.to_datetime(df['Date'], format='%m/%d/%y') - pd.to_timedelta(offset, unit='D')
df.insert(2, 'WeekStartingSat', most_recent_saturday)
df.to_csv('vending.csv', index=False)
"""

In [181]:
import pandas as pd

df = pd.read_csv('vending.csv')
df

,TransNum,Date,WeekStartingSat,WeekNum,Location,Product,Coil,Price,Quantity,Total,...,HealthyMarketing_True,Payment_Cash,Payment_Credit,DayOfWeek_Sat,DayOfWeek_Sun,DayOfWeek_Mon,DayOfWeek_Tue,DayOfWeek_Wed,DayOfWeek_Thu,DayOfWeek_Fri
0,0,1/1/22,1/1/22,0,Mall 1,Red Bull - Energy Drink - Sugar Free,148,3.5,1,3.5,...,1,0,1,1,0,0,0,0,0,0
1,1,1/1/22,1/1/22,0,Mall 1,Red Bull - Energy Drink - Sugar Free,148,3.5,1,3.5,...,1,0,1,1,0,0,0,0,0,0
2,2,1/1/22,1/1/22,0,Mall 1,Takis - Hot Chilli Pepper & Lime,123,1.5,1,1.5,...,0,0,1,1,0,0,0,0,0,0
3,3,1/1/22,1/1/22,0,Mall 1,Takis - Hot Chilli Pepper & Lime,123,1.5,1,1.5,...,0,0,1,1,0,0,0,0,0,0
4,4,1/1/22,1/1/22,0,Mall 1,Red Bull - Energy Drink - Sugar Free,148,3.5,1,3.5,...,1,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9374,9374,12/30/22,12/24/22,51,Factory,Doritos Nacho Cheese,112,1.5,1,1.5,...,0,1,0,0,0,0,0,0,0,1
9375,9375,12/31/22,12/31/22,52,Mall 1,Poland Springs Water,143,1.5,1,1.5,...,1,1,0,1,0,0,0,0,0,0
9376,9376,12/31/22,12/31/22,52,Mall 2,Robert Irvine's - Fit Crunch - Chocolate Pea,137,2.0,1,2.0,...,1,1,0,1,0,0,0,0,0,0
9377,9377,12/31/22,12/31/22,52,Mall 1,Poland Springs Water,143,1.5,1,1.5,...,1,1,0,1,0,0,0,0,0,0


In [182]:
df1 = df[['WeekStartingSat', 'Location', 'Total', 'Quantity']].groupby(['WeekStartingSat', 'Location']).sum()
df1['AvgPriceItemSold'] = df1['Total']/df1['Quantity']

df2 = df[['WeekStartingSat', 'Location',
               'Type_Drink', 'Type_Food',
               'Category_Carbonated', 'Category_Non Carbonated', 'Category_Salty', 'Category_Sweet', 'Category_Water',
               'HealthyMarketing_False', 'HealthyMarketing_True',
               'Payment_Cash','Payment_Credit',
               'DayOfWeek_Sat', 'DayOfWeek_Sun', 'DayOfWeek_Mon', 'DayOfWeek_Tue', 'DayOfWeek_Wed', 'DayOfWeek_Thu', 'DayOfWeek_Fri'
               ]].groupby(['WeekStartingSat', 'Location']).mean()

df_week = pd.merge(df1, df2, left_index=True, right_index=True)
df_week


Total  Quantity  AvgPriceItemSold  Type_Drink  \
WeekStartingSat Location                                                   
1/1/22          Factory    58.50        35          1.671429    0.228571   
                Mall 1     23.00         9          2.555556    0.666667   
                Mall 2     17.75         9          1.972222    0.625000   
                Office     26.75        13          2.057692    0.153846   
1/15/22         Factory   178.25        95          1.876316    0.463158   
...                          ...       ...               ...         ...   
9/3/22          Factory   142.00        75          1.893333    0.698630   
                Library   114.75        50          2.295000    0.240000   
                Mall 1     12.50         6          2.083333    0.400000   
                Mall 2     28.00        16          1.750000    0.266667   
                Office     48.50        28          1.732143    0.000000   

                          Type_Food  Category_Carbonated  \
WeekStartingSat Location                                   
1/1/22          Factory    0.771429             0.171429   
                Mall 1     0.333333             0.666667   
                Mall 2     0.375000             0.125000   
                Office     0.846154             0.000000   
1/15/22         Factory    0.536842             0.347368   
...                             ...                  ...   
9/3/22          Factory    0.301370             0.616438   
                Library    0.760000             0.120000   
                Mall 1     0.600000             0.000000   
                Mall 2     0.733333             0.066667   
                Office     1.000000             0.000000   

                          Category_Non Carbonated  Category_Salty  \
WeekStartingSat Location                                            
1/1/22          Factory                  0.057143        0.342857   
                Mall 1                   0.000000        0.333333   
                Mall 2                   0.250000        0.250000   
                Office                   0.153846        0.692308   
1/15/22         Factory                  0.115789        0.357895   
...                                           ...             ...   
9/3/22          Factory                  0.082192        0.232877   
                Library                  0.120000        0.500000   
                Mall 1                   0.400000        0.400000   
                Mall 2                   0.066667        0.200000   
                Office                   0.000000        0.571429   

                          Category_Sweet  Category_Water  ...  \
WeekStartingSat Location                                  ...   
1/1/22          Factory         0.428571        0.000000  ...   
                Mall 1          0.000000        0.000000  ...   
                Mall 2          0.125000        0.250000  ...   
                Office          0.153846        0.000000  ...   
1/15/22         Factory         0.178947        0.000000  ...   
...                                  ...             ...  ...   
9/3/22          Factory         0.068493        0.000000  ...   
                Library         0.260000        0.000000  ...   
                Mall 1          0.200000        0.000000  ...   
                Mall 2          0.533333        0.133333  ...   
                Office          0.428571        0.000000  ...   

                          HealthyMarketing_True  Payment_Cash  Payment_Credit  \
WeekStartingSat Location                                                        
1/1/22          Factory                0.485714      0.942857        0.057143   
                Mall 1                 0.444444      0.111111        0.888889   
                Mall 2                 0.750000      0.875000        0.125000   
                Office                 0.846154      0.769231        0.230769   
1/15/22         Factory    

In [183]:
import pandas as pd
import numpy as np

df1 = df[['WeekStartingSat', 'Location', 'Total', 'Quantity']].groupby(['WeekStartingSat', 'Location']).sum()
df1['AvgPriceItemSold'] = df1['Total']/df1['Quantity']

df2 = df[['WeekStartingSat', 'Location',
               'Type_Drink', 'Type_Food',
               'Category_Carbonated', 'Category_Non Carbonated', 'Category_Salty', 'Category_Sweet', 'Category_Water',
               'HealthyMarketing_False', 'HealthyMarketing_True',
               'Payment_Cash','Payment_Credit',
               'DayOfWeek_Sat', 'DayOfWeek_Sun', 'DayOfWeek_Mon', 'DayOfWeek_Tue', 'DayOfWeek_Wed', 'DayOfWeek_Thu', 'DayOfWeek_Fri'
               ]].groupby(['WeekStartingSat', 'Location']).mean()

df_week = pd.merge(df1, df2, left_index=True, right_index=True)

# Generate weekly dates for 2022
dates = pd.date_range(start='2022-01-01', end='2022-12-31', freq='W-SAT').strftime('%-m/%-d/%y')

# Create a list of store names
locations = ["Factory", "Library", "Office", "Mall 1", "Mall 2"]

# Create a MultiIndex from the Cartesian product of dates and stores
index = pd.MultiIndex.from_product([dates, locations], names=['WeekStartingSat', 'Location'])

# Create a DataFrame with the multiindex
data = pd.DataFrame(np.nan, index=index, columns=['Sales'])


df_week = pd.merge(data, df_week, how='outer', left_index=True, right_index=True)
df_week = df_week.drop(columns='Sales')

df_week['Total'] = df_week['Total'].fillna(0)
df_week['Quantity'] = df_week['Quantity'].fillna(0)

df_week = df_week.reset_index()
df_week['WeekStartingSat'] = pd.to_datetime(df_week['WeekStartingSat'], format='%m/%d/%y')
df_week = df_week.sort_values(by=['WeekStartingSat', 'Location'])

df_week.to_csv('vending_revenue.csv', index=False)

In [184]:
df_week

,WeekStartingSat,Location,Total,Quantity,AvgPriceItemSold,Type_Drink,Type_Food,Category_Carbonated,Category_Non Carbonated,Category_Salty,...,HealthyMarketing_True,Payment_Cash,Payment_Credit,DayOfWeek_Sat,DayOfWeek_Sun,DayOfWeek_Mon,DayOfWeek_Tue,DayOfWeek_Wed,DayOfWeek_Thu,DayOfWeek_Fri
0,2022-01-01,Factory,58.50,35.0,1.671429,0.228571,0.771429,0.171429,0.057143,0.342857,...,0.485714,0.942857,0.057143,0.000000,0.000,0.228571,0.342857,0.171429,0.142857,0.114286
1,2022-01-01,Library,0.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-01,Mall 1,23.00,9.0,2.555556,0.666667,0.333333,0.666667,0.000000,0.333333,...,0.444444,0.111111,0.888889,0.555556,0.000,0.111111,0.111111,0.111111,0.111111,0.000000
3,2022-01-01,Mall 2,17.75,9.0,1.972222,0.625000,0.375000,0.125000,0.250000,0.250000,...,0.750000,0.875000,0.125000,0.000000,0.125,0.125000,0.250000,0.000000,0.250000,0.250000
4,2022-01-01,Office,26.75,13.0,2.057692,0.153846,0.846154,0.000000,0.153846,0.692308,...,0.846154,0.769231,0.230769,0.000000,0.000,0.384615,0.076923,0.153846,0.307692,0.076923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,2022-12-31,Factory,0.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,2022-12-31,Library,0.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,2022-12-31,Mall 1,3.00,2.0,1.500000,1.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,1.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000
93,2022-12-31,Mall 2,3.50,2.0,1.750000,0.500000,0.500000,0.500000,0.000000,0.000000,...,1.000000,1.000000,0.000000,1.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
# df.to_csv('vending.csv')

# Make dataset for basket analysis

In [252]:
df = pd.read_csv('vending.csv')
df

,TransNum,Date,WeekStartingSat,WeekNum,Location,Product,Coil,Price,Quantity,Total,...,HealthyMarketing_True,Payment_Cash,Payment_Credit,DayOfWeek_Sat,DayOfWeek_Sun,DayOfWeek_Mon,DayOfWeek_Tue,DayOfWeek_Wed,DayOfWeek_Thu,DayOfWeek_Fri
0,0,1/1/22,1/1/22,0,Mall 1,Red Bull - Energy Drink - Sugar Free,148,3.5,1,3.5,...,1,0,1,1,0,0,0,0,0,0
1,1,1/1/22,1/1/22,0,Mall 1,Red Bull - Energy Drink - Sugar Free,148,3.5,1,3.5,...,1,0,1,1,0,0,0,0,0,0
2,2,1/1/22,1/1/22,0,Mall 1,Takis - Hot Chilli Pepper & Lime,123,1.5,1,1.5,...,0,0,1,1,0,0,0,0,0,0
3,3,1/1/22,1/1/22,0,Mall 1,Takis - Hot Chilli Pepper & Lime,123,1.5,1,1.5,...,0,0,1,1,0,0,0,0,0,0
4,4,1/1/22,1/1/22,0,Mall 1,Red Bull - Energy Drink - Sugar Free,148,3.5,1,3.5,...,1,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9374,9374,12/30/22,12/24/22,51,Factory,Doritos Nacho Cheese,112,1.5,1,1.5,...,0,1,0,0,0,0,0,0,0,1
9375,9375,12/31/22,12/31/22,52,Mall 1,Poland Springs Water,143,1.5,1,1.5,...,1,1,0,1,0,0,0,0,0,0
9376,9376,12/31/22,12/31/22,52,Mall 2,Robert Irvine's - Fit Crunch - Chocolate Pea,137,2.0,1,2.0,...,1,1,0,1,0,0,0,0,0,0
9377,9377,12/31/22,12/31/22,52,Mall 1,Poland Springs Water,143,1.5,1,1.5,...,1,1,0,1,0,0,0,0,0,0


In [253]:
# Count unique products by location
for loc in df['Location'].unique():
    df_sub = df[df['Location']==loc]
    print(loc, len(df_sub['Product']), len(df_sub['Product'].unique()))

Mall 1 751 64
Mall 2 928 70
Office 1040 66
Factory 3598 76
Library 3062 98


In [254]:
# Make list of baskets where each basket is the unique transactions from a day
dff = df[df['Location']=='Factory']
dff = dff[['Date', 'Product']]

basket_list = []

for day in dff['Date'].unique():
    basket = dff[dff['Date']==day]
    basket = basket.drop_duplicates()
    basket = set(basket['Product'])
    basket_list.append(basket)

basket_list

[{'Monster Energy Original',
  'Skinny Pop Popcorn',
  'Snapple Diet Tea - Lemon',
  'Stretch Island Fruit Leathers Snacks - Variet'},
 {'Cheetos - Fleming Hot Crunchy',
  'Jacks Links Bar Beef Strip',
  'KitKat - Crisp Wafers',
  'Nutter Butter Bites',
  'Oreo Mini',
  'Pure Organic Fruit Bar - Strawberry Banana',
  'Spindrift - Sparkling Water  Lime'},
 {'Good Health Veggie Stix - Zesty Ranch',
  'Jack Links Beef Steak Original',
  'Nutter Butter Bites',
  'Pure Organic Fruit Bar - Strawberry Banana',
  'Skinny Pop Popcorn'},
 {'Cheetos Baked - Flaming Hot',
  'KitKat - Crisp Wafers',
  'Mini Chips Ahoy',
  'Newtons Fig Bar'},
 {'Cheetos Baked - Flaming Hot', 'KitKat - Crisp Wafers', 'Oreo Mini'},
 {'Takis - Hot Chilli Pepper & Lime'},
 {'Mini Chips Ahoy'},
 {'Mini Chips Ahoy', 'Takis - Hot Chilli Pepper & Lime'},
 {'Chum Fruit Bites - Mango/Strawberry'},
 {'Cheetos Baked - Flaming Hot',
  'Chesters Fries Flaming hot',
  'Coca Cola - Zero Sugar',
  'KitKat - Crisp Wafers',
  'Mini Ch

In [207]:
import numpy as np

# Extract unique items from the grocery list
unique_items = list(set.union(*basket_list))

# Create an empty matrix to store the frequencies
frequency_matrix = np.zeros((len(unique_items), len(unique_items)), dtype=int)

# Iterate over each set in the grocery list
for item_set in basket_list:
    # Convert the set to a list for indexing purposes
    items = list(item_set)

    # Increment the frequency count for each pair of items in the set
    for i in range(len(items)):
        for j in range(i + 1, len(items)):
            item1 = items[i]
            item2 = items[j]

            # Find the indices of the items in the unique items list
            item1_index = unique_items.index(item1)
            item2_index = unique_items.index(item2)

            # Increment the corresponding entry in the frequency matrix
            frequency_matrix[item1_index][item2_index] += 1
            frequency_matrix[item2_index][item1_index] += 1

# Create a pandas DataFrame from the frequency matrix
frequency_df = pd.DataFrame(frequency_matrix, index=unique_items, columns=unique_items)
frequency_df.index.name = 'ProductFrequency'

# Display the frequency matrix
frequency_df #.to_csv('vending_frequencies.csv')

,Miss Vickie's Potato Chip - Sea Salt & Vinega,SunChips Multigrain - Original,Chesters Fries Flaming hot,Nutri Grain Apple Cinnamon,CheezIt - Original,Takis FUGEO mini,Sunkist Soda - Orange,SunChips Multigrain - Salsa,Pop Corners - Spicy Queso,Ruffles Baked - Cheddar & Sour Cream,...,Snapple Diet Tea - Lemon,Good Health Veggie Stix - Sea Salt,Nature's Bakery Fig Bar - Raspberry,Doritos Nacho Cheese,Takis - Hot Chilli Pepper & Lime,Miss Vickie's Potato Chip - Smokehouse BBQ,Snapple Tea - Raspberry,Oreo Mini,Ritz Bits Cheese,Coca Cola - Regular
ProductFrequency,,,,,,,,,,,,,,,,,,,,,
Miss Vickie's Potato Chip - Sea Salt & Vinega,0,0,0,0,0,0,1,0,0,0,...,2,1,0,0,0,0,1,0,0,0
SunChips Multigrain - Original,0,0,2,0,0,0,6,2,0,1,...,0,0,5,2,0,0,3,0,1,3
Chesters Fries Flaming hot,0,2,0,5,0,2,27,3,0,3,...,2,2,10,2,0,1,14,4,7,14
Nutri Grain Apple Cinnamon,0,0,5,0,0,0,15,0,5,0,...,5,0,0,0,0,0,6,2,2,3
CheezIt - Original,0,0,0,0,0,0,1,2,0,0,...,1,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Miss Vickie's Potato Chip - Smokehouse BBQ,0,0,1,0,0,1,0,0,0,0,...,1,2,0,0,0,0,0,0,3,0
Snapple Tea - Raspberry,1,3,14,6,0,6,34,3,3,3,...,10,1,18,4,0,0,0,5,4,23
Oreo Mini,0,0,4,2,0,0,11,1,0,0,...,3,0,2,0,0,0,5,0,4,2


# Market basket analysis in Python

In [283]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Convert the grocery list to a one-hot encoded transaction format
te = TransactionEncoder()
te_ary = te.fit_transform(basket_list)
df = pd.DataFrame(te_ary, columns=te.columns_)

# Apply the Apriori algorithm to find frequent itemsets
frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True, max_len=2)

# Generate association rules
association_rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1)

# Display the association rules
print("\nAssociation Rules:")
association_rules


Association Rules:


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Cheetos - Crunchy),(Cheetos - Fleming Hot Crunchy),0.078231,0.261905,0.027211,0.347826,1.328063,0.006722,1.131746,0.267989
1,(Cheetos - Fleming Hot Crunchy),(Cheetos - Crunchy),0.261905,0.078231,0.027211,0.103896,1.328063,0.006722,1.028640,0.334677
2,(Chesters Fries Flaming hot),(Cheetos - Crunchy),0.156463,0.078231,0.017007,0.108696,1.389414,0.004767,1.034180,0.332258
3,(Cheetos - Crunchy),(Chesters Fries Flaming hot),0.078231,0.156463,0.017007,0.217391,1.389414,0.004767,1.077853,0.304059
4,(Coca Cola - Regular),(Cheetos - Crunchy),0.265306,0.078231,0.034014,0.128205,1.638796,0.013258,1.057323,0.530556
...,...,...,...,...,...,...,...,...,...,...
1109,(SunChips Multigrain - Harvest Cheddar),(SunChips Multigrain - Salsa),0.098639,0.057823,0.037415,0.379310,6.559838,0.031711,1.517952,0.940309
1110,(Sunkist Soda - Orange),(SunChips Multigrain - Harvest Cheddar),0.445578,0.098639,0.044218,0.099237,1.006054,0.000266,1.000663,0.010854
1111,(SunChips Multigrain - Harvest Cheddar),(Sunkist Soda - Orange),0.098639,0.445578,0.044218,0.448276,1.006054,0.000266,1.004889,0.006676
1112,(SunChips Multigrain - Salsa),(Sunkist Soda - Orange),0.057823,0.445578,0.030612,0.529412,1.188145,0.004848,1.178146,0.168071


In [284]:
# Order by lift descending
association_rules = association_rules.sort_values(by='lift', ascending=False)
association_rules = association_rules.drop_duplicates(subset='antecedents', keep='first')
association_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
979,(Pop Corners - Kettle Corn),(Quaker Caramel Rice Crisps),0.017007,0.017007,0.010204,0.600000,35.280000,0.009915,2.457483,0.988466
978,(Quaker Caramel Rice Crisps),(Pop Corners - Kettle Corn),0.017007,0.017007,0.010204,0.600000,35.280000,0.009915,2.457483,0.988466
1067,(Snapple Lemonade - watermelon),(Snapple Lemonade - pineapple),0.013605,0.023810,0.010204,0.750000,31.500000,0.009880,3.904762,0.981609
1066,(Snapple Lemonade - pineapple),(Snapple Lemonade - watermelon),0.023810,0.013605,0.010204,0.428571,31.500000,0.009880,1.726190,0.991870
328,(Skinny Pop Popcorn),(CheezIt - Original),0.040816,0.017007,0.010204,0.250000,14.700000,0.009510,1.310658,0.971631
...,...,...,...,...,...,...,...,...,...,...
761,(KitKat - Crisp Wafers),(Quaker Caramel Rice Crisps),0.357143,0.017007,0.013605,0.038095,2.240000,0.007532,1.021924,0.861111
353,(Munchies Snack mix Flaming Hot),(Chesters Fries Flaming hot),0.030612,0.156463,0.010204,0.333333,2.130435,0.005414,1.265306,0.547368
758,(Pure Organic Pineapple Passionfruit),(KitKat - Crisp Wafers),0.013605,0.357143,0.010204,0.750000,2.100000,0.005345,2.571429,0.531034
385,(Chum Fruit Bites - Mango/Strawberry),(Coca Cola - Zero Sugar),0.027211,0.285714,0.013605,0.500000,1.750000,0.005831,1.428571,0.440559


In [306]:
# Make a dictionary of the top recommendation
dictionary = {}
for index, row in association_rules.iterrows():
    key = next(iter(row['antecedents']))  # Extract item from frozenset
    value = next(iter(row['consequents']))
    dictionary[key] = value
dictionary

{'Pop Corners - Kettle Corn': 'Quaker Caramel Rice Crisps',
 'Quaker Caramel Rice Crisps': 'Pop Corners - Kettle Corn',
 'Snapple Lemonade - watermelon': 'Snapple Lemonade - pineapple',
 'Snapple Lemonade - pineapple': 'Snapple Lemonade - watermelon',
 'Skinny Pop Popcorn': 'CheezIt - Original',
 'CheezIt - Original': 'Skinny Pop Popcorn',
 'Ritz Bits Cheese': "Miss Vickie's Potato Chip - Smokehouse BBQ",
 "Miss Vickie's Potato Chip - Smokehouse BBQ": 'Ritz Bits Cheese',
 "Miss Vickie's Potato Chip - Lime & Cracked Pe": 'Ritz Bits Cheese',
 'Oreo Single Server 6 ct': 'CheezIt - Original',
 "Miss Vickie's Potato Chip - Jalapeno": 'Mini Chips Ahoy',
 'Mini Chips Ahoy': "Miss Vickie's Potato Chip - Jalapeno",
 'Cheetos Baked - Flaming Hot': "Miss Vickie's Potato Chip - Jalapeno",
 'Cheetos Baked - Crunchy Cheese': 'Lays - Barbecue',
 'Lays - Barbecue': 'Cheetos Baked - Crunchy Cheese',
 'Jacks Links Bar Beef Strip': 'Good Health Veggie Stix - Sea Salt',
 'Good Health Veggie Stix - Sea Sal